In [12]:
# Automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import model_setup as ms
c = ms.ModelConfig('./config/UKR.toml')
# c = ms.ModelConfig('./config/CHN_2024_EFC.toml')

from model_setup.solution_index import SolutionIndexCreator
from model_setup.capacity_setup import CapacitySetup
from model_setup.transmission_setup import TransmissionSetup
from model_setup.load_setup import LoadSetup


In [14]:
ts = TransmissionSetup(c)

In [15]:
ls = LoadSetup(c)

initialising class
class initialised


In [9]:
df = ls._read_manual_demand_data()

In [11]:
### Split hourly demand using split in snapshots
import openpyxl as xl 
import os
  



modelled_regs = c.list_neighbouring_countries

demand_sector_dict = {}
demand_by_reg_df = pd.DataFrame(None)

##

folder_path = c.cfg['load']['load_data_path']
demand_file_paths = [f for f in os.listdir(folder_path) if ('demand_snapshot' in f) &('$' not in f)]

for path in demand_file_paths:
    
    full_path = os.path.join(folder_path, path)
    # load the workbook 
    wb = xl.load_workbook(full_path)     
    sheet_names = [ s for s in wb.sheetnames if s != 'UKR' ] 
    demand_by_sector_df = pd.DataFrame(None)
    
    
    for reg in sheet_names:
    
        demand_df = pd.read_excel(full_path, sheet_name=reg, header=7, usecols='A:Z', nrows = 52)
        keep_rows = [15, 18, 44, 45, 49, 50, 52] ## Losses = 15, removed. keep rows for sector totals and losses . other own-use is explicit in PLEXOS (double-check)
        rename_cols = {'I. INDUSTRY:':'Industrial', 'II. AGRICULTURAL CONSUMERS':'Agricultural',
            'III. TRANSPORT':'Transport', 'IV. CONSTRUCTION':'Construction', 'V. UTILITIES':'Utilities',
            'VI. LIGHT, HOUSEHOLD AND OTHER SP.':'Residential'}

        demand_df = demand_df[demand_df.line.isin(keep_rows)].drop(columns=['line']).set_index('consumer groups').T
        demand_df = demand_df.rename(columns=rename_cols)
            #####
        demand_by_sector_df[reg] = demand_df.sum(axis=0)
        demand_by_sector_df = demand_by_sector_df.rename_axis('Sector')
#         demand_by_sector_df = demand_by_sector_df/demand_by_sector_df.sum(axis=0)*100
#         demand_by_sector_df = demand_by_sector_df.round(1)
        
    
        demand_by_reg_df[path.split('_')[-1].split('.')[0]] = demand_by_sector_df.sum(axis=0)
        demand_sector_dict[path.split('_')[-1].split('.')[0]] = demand_by_sector_df

    demand_sector_dict['2021_prime'] = demand_sector_dict['2021'].copy()
    #######
    demand_sector_dict['2021_prime'].loc['Industrial', 'EOR'] = demand_sector_dict['2021_prime'].loc['Industrial', 'EOR']*0.5
    demand_sector_dict['2021_prime'].loc['Agricultural', 'EOR'] = demand_sector_dict['2021_prime'].loc['Agricultural', 'EOR']*0.5
    demand_sector_dict['2021_prime'].loc['Transport', 'EOR'] = demand_sector_dict['2021_prime'].loc['Transport', 'EOR']*0.5
    demand_sector_dict['2021_prime'].loc['Construction', 'EOR'] = demand_sector_dict['2021_prime'].loc['Construction', 'EOR']*0.5
    demand_sector_dict['2021_prime'].loc['Utilities', 'EOR'] = demand_sector_dict['2021_prime'].loc['Utilities', 'EOR']*0.5
    demand_sector_dict['2021_prime'].loc['Residential', 'EOR'] = demand_sector_dict['2021_prime'].loc['Residential', 'EOR']*0.5

    #######
    demand_by_reg_df['2021_prime'] = demand_sector_dict['2021_prime'].sum(axis=0)

    ### Demand percentage relative to 2021 so we can use the UKRAINE national demand as the basis (this could be improved)
    ### This is used for scaling to obtain full profile of UKRAINE demand by region without all profiles.
    ### Can be compared vs. actuals (e.g. snapshot we actually have)
    ### The limitation is that 2022 scaling factor is based on summer, therefore we should obtain a scaling factor for 2022 to correct this
    demand_by_reg_pc_df = demand_by_reg_df/demand_by_reg_df.sum()['2021']
    #######
    demand_by_reg_df.loc['EOR', '2023'] = demand_by_reg_df['2023'].sum()*demand_by_reg_pc_df.loc['EOR', '2021_prime']
    demand_by_reg_df.loc['EOR', '2022'] = demand_by_reg_df['2022'].sum()*demand_by_reg_pc_df.loc['EOR', '2021_prime']
    demand_by_reg_df = demand_by_reg_df.drop(columns=['2021_prime'])
    demand_by_reg_pc_df = demand_by_reg_df/demand_by_reg_df.sum()['2021']
    demand_by_reg_pc_df = demand_by_reg_pc_df[demand_by_reg_pc_df.columns.sort_values()]

KeyError: 'load_data_path'

In [ ]:
ts = TransmissionSetup(c)
ts.calculate_tx_capacities()

In [ ]:
from riselib.gis import get_country_gdf

In [ ]:
x = get_country_gdf('UKR')

In [ ]:
x.loc[:,'GID_0'] = 'UKR'

In [ ]:
border_names = ts.border_df.name.unique()
lines_by_border = ts.gdf_tx.copy()
lines_by_border

lines_by_border.index = lines_by_border.index.astype(int)


In [ ]:
sil_by_voltage = ts.df_tx_cap_by_v.set_index('Voltage')['SIL']
sil_by_voltage.loc[lines_by_border.index.astype(int)]*lines_by_border.T


In [ ]:
cs.setup_capacity()
cs.setup_plant_parameters()
cs.export_plant_parameters()